In [61]:
import pandas as pd

past = pd.read_excel('../asset/past.xlsx', engine='openpyxl')
present = pd.read_excel('../asset/present.xlsx', engine='openpyxl')

In [62]:
past.fillna('', inplace=True)
present.fillna('', inplace=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_54503/1969576918.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  past.fillna('', inplace=True)
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_54503/1969576918.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  present.fillna('', inplace=True)


In [63]:
condition = present['편집자'] == '웅숭코전당포'
present = present[condition]
present.reset_index(inplace=True, drop=True)
present['어깨번호'] = present['어깨번호'].astype('Int32')
present['제공처ID'] = present['제공처ID'].astype('Int32')
present

,엔트리ID,표제어,어깨번호,표제어종류,제공처ID,편집자
0,00d1ee2f897c43e899107915a7a7308b,가락 바로잡는 집에 가져다가 세워 놨다 와도 좀 낫다,0,속담,8152,웅숭코전당포
1,2110126e34b94e95a2348c44b2d0f3c2,비교(가) 안 되다〔되지 않다〕,0,숙어,5106,웅숭코전당포
2,28a6868d81134c6aa26fe118ca0cda77,나 죽었소 하다,0,숙어,5131,웅숭코전당포
3,2aeceea596e245499efdd93993780d23,걸신이 들린 듯이,0,숙어,5138,웅숭코전당포
4,3131dd3acec543ee8bc3ce41b3dcb156,생콩〔날콩〕 씹은 상판,0,숙어,4218,웅숭코전당포
...,...,...,...,...,...,...
411505,f1a528b9023a42a3ae583b5205071cfd,유엔에이치시알,0,단어,466341,웅숭코전당포
411506,f2dda3e8636846fea325b696400cce1c,아이알비엠,0,단어,466304,웅숭코전당포
411507,f3ddf94a04414907a1e886fb638ece42,알에이치 인자,0,단어,466289,웅숭코전당포
411508,f9d321279e8b4085bb38d4af7f47e5b8,전령 알엔에이,0,단어,466344,웅숭코전당포


In [64]:
# past_main = past[[
#     '엔트리ID', '표제어', '어깨번호', '원어 멤버ID', '차별 표현', '차별 표현 링크', '뜻풀이 ID',
#     '예문 ID', '활용형 ID'
# ]]
# past_related = past[[
#     '연관어 표제어 ID', '연관단어 ID', '연관어 뜻 ID', '연관단어 ID.1'
# ]]

In [65]:
pm_valid = past[past['엔트리ID'] != ''].copy()

merged = pd.merge(
    pm_valid,
    present,
    on='엔트리ID',
    how='left',
    suffixes=('_old', '')
)

past = past.reset_index(drop=True)
merged = merged.reset_index(drop=True)

cols_to_update = ['표제어','어깨번호','표제어종류','제공처ID','편집자']
past.loc[past['엔트리ID'] != '', cols_to_update] = merged[cols_to_update].values

In [66]:
past.columns

Index(['엔트리ID', '표제어', '어깨번호', '원어 멤버ID', '차별 표현', '차별 표현 링크', '뜻풀이 ID',
       '예문 ID', '연관단어 ID', '연관어 뜻 ID', '연관어 표제어 ID', '활용형 ID', '연관단어 ID.1',
       '연관어 표제어 ID.1', '연관어', '표제어종류', '제공처ID', '편집자'],
      dtype='object')

In [67]:
past = past[[
    '엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', 
    '차별 표현', '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID', '연관어 표제어 ID', '연관단어 ID.1', '연관어'
]]

past.fillna('', inplace=True)

In [68]:
past.to_csv('../asset/ohw_id.csv', index=False)

In [69]:
past.columns

Index(['엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', '차별 표현',
       '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID',
       '연관어 표제어 ID', '연관단어 ID.1', '연관어'],
      dtype='object')

In [70]:
past_mean = past[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '차별 표현',
    '차별 표현 링크', '뜻풀이 ID'
]]
past_mean = past_mean[(past_mean != '').any(axis=1)]
past_mean['뜻풀이번호'] = int(1)
past_mean.reset_index(inplace=True, drop=True)
LEN = len(past_mean)
index = 2
for i in range(LEN):
    if past_mean.loc[i, '엔트리ID'] != '':
        index = 2
    else:
        past_mean.loc[i, '뜻풀이번호'] = index
        index += 1

past_mean = past_mean[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '뜻풀이번호', '차별 표현',
    '차별 표현 링크', '뜻풀이 ID'
]]

past_mean

,엔트리ID,원어 멤버ID,표제어,어깨번호,표제어종류,뜻풀이번호,차별 표현,차별 표현 링크,뜻풀이 ID
0,04aa3a701a0445d4adb7a47c306d03d7,06bb8a984d73451bab161583833af5ec,디지털 방송,0,단어,1,,,db68fb14c23c4e10b581b1093d2d18e6
1,04e90ba76686402bb7c9a10c6a8748fa,b53df7b89f2043228735226c8d0b0187,반당,3,단어,1,,,566f088e0cfb4e679e05745504031dad
2,,,,,,2,,,61624063641a4b32a39cefe2e18b44b8
3,0568c5702e4842e88e5fbce9d0e21045,770eefa8c8884d70aec1b51962504057,설문하다,2,단어,1,,,559e15e8d086477581177891304de72a
4,091131a76093407fb77701cad7877766,8a97ee8f1d9642fcba40ee869ec2effa,티금새기,0,단어,1,,,09f8dbbc21d547c999ffa2157109e6a0
...,...,...,...,...,...,...,...,...,...
485387,f816cf75d12647e5b91bd8108e81fb14,,나무를 보고 숲을 보지 못한다,0,속담,1,,,
485388,f8b2b32364f44380a1330d58f2db94ec,,이웃집 개도 부르면 온다,0,속담,1,,,
485389,fa35d48b656a4b8596aeaeb3c7c2487b,,피는 물보다 진하다,0,속담,1,,,
485390,fa6ab593ea794835a16a282ba9045b04,,구두장이 셋이면 제갈량의 꾀를 이긴다,0,속담,1,,,


In [71]:
past_mean.to_csv('../asset/ohw_id_mean.csv', index=False)

In [72]:
past_mean.columns, past.columns

(Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '뜻풀이번호', '차별 표현',
        '차별 표현 링크', '뜻풀이 ID'],
       dtype='object'),
 Index(['엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', '차별 표현',
        '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID',
        '연관어 표제어 ID', '연관단어 ID.1', '연관어'],
       dtype='object'))

In [73]:
past_exam = past[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '예문 ID'
]]
past_exam = past_exam[(past_exam != '').any(axis=1)]
past_exam['예문번호'] = int(1)
past_exam.reset_index(inplace=True, drop=True)
past_exam = past_exam[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '예문번호', '예문 ID'
]]
LEN = len(past_exam)
index = 2
for i in range(LEN):
    if past_exam.loc[i, '엔트리ID'] != '':
        index = 2
    else:
        past_exam.loc[i, '예문번호'] = index
        index += 1
past_exam

,엔트리ID,원어 멤버ID,표제어,어깨번호,표제어종류,예문번호,예문 ID
0,04aa3a701a0445d4adb7a47c306d03d7,06bb8a984d73451bab161583833af5ec,디지털 방송,0,단어,1,
1,04e90ba76686402bb7c9a10c6a8748fa,b53df7b89f2043228735226c8d0b0187,반당,3,단어,1,2f889b595c584cd08adf3fbe1be12ef0
2,0568c5702e4842e88e5fbce9d0e21045,770eefa8c8884d70aec1b51962504057,설문하다,2,단어,1,
3,091131a76093407fb77701cad7877766,8a97ee8f1d9642fcba40ee869ec2effa,티금새기,0,단어,1,
4,0c94e6f5651e49939e5318345e774815,3da117d670014591bed17c5234c3ab35,수정하다,4,단어,1,
...,...,...,...,...,...,...,...
536726,f816cf75d12647e5b91bd8108e81fb14,,나무를 보고 숲을 보지 못한다,0,속담,1,
536727,f8b2b32364f44380a1330d58f2db94ec,,이웃집 개도 부르면 온다,0,속담,1,
536728,fa35d48b656a4b8596aeaeb3c7c2487b,,피는 물보다 진하다,0,속담,1,
536729,fa6ab593ea794835a16a282ba9045b04,,구두장이 셋이면 제갈량의 꾀를 이긴다,0,속담,1,


In [74]:
past_exam.to_csv('../asset/ohw_id_exam.csv', index=False)

In [75]:
past.columns, past_exam.columns

(Index(['엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', '차별 표현',
        '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID',
        '연관어 표제어 ID', '연관단어 ID.1', '연관어'],
       dtype='object'),
 Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '예문번호', '예문 ID'], dtype='object'))

In [76]:
past_use = past[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '활용형 ID'
]]
past_use = past_use[(past_use != '').any(axis=1)]
past_use['활용번호'] = int(1)
past_use.reset_index(inplace=True, drop=True)
past_use = past_use[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '활용번호', '활용형 ID'
]]
LEN = len(past_use)
index = 2
for i in range(LEN):
    if past_use.loc[i, '엔트리ID'] != '':
        index = 2
    else:
        past_use.loc[i, '활용번호'] = index
        index += 1
past_use

,엔트리ID,원어 멤버ID,표제어,어깨번호,표제어종류,활용번호,활용형 ID
0,04aa3a701a0445d4adb7a47c306d03d7,06bb8a984d73451bab161583833af5ec,디지털 방송,0,단어,1,
1,04e90ba76686402bb7c9a10c6a8748fa,b53df7b89f2043228735226c8d0b0187,반당,3,단어,1,
2,0568c5702e4842e88e5fbce9d0e21045,770eefa8c8884d70aec1b51962504057,설문하다,2,단어,1,c4054f085df141bd8bfa275c1e5c093f
3,,,,,,2,c7e8bd386fb6415b9b57041a634ede8d
4,,,,,,3,3e1fe36dbed1462d8b568a16fa5aa25a
...,...,...,...,...,...,...,...
525272,f816cf75d12647e5b91bd8108e81fb14,,나무를 보고 숲을 보지 못한다,0,속담,1,
525273,f8b2b32364f44380a1330d58f2db94ec,,이웃집 개도 부르면 온다,0,속담,1,
525274,fa35d48b656a4b8596aeaeb3c7c2487b,,피는 물보다 진하다,0,속담,1,
525275,fa6ab593ea794835a16a282ba9045b04,,구두장이 셋이면 제갈량의 꾀를 이긴다,0,속담,1,


In [77]:
past_use.to_csv('../asset/ohw_id_use.csv', index=False)

In [78]:
past.columns, past_use.columns

(Index(['엔트리ID', '표제어', '어깨번호', '표제어종류', '제공처ID', '편집자', '원어 멤버ID', '차별 표현',
        '차별 표현 링크', '뜻풀이 ID', '예문 ID', '활용형 ID', '연관단어 ID', '연관어 뜻 ID',
        '연관어 표제어 ID', '연관단어 ID.1', '연관어'],
       dtype='object'),
 Index(['엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '활용번호', '활용형 ID'], dtype='object'))

In [84]:
past_rel = past[[
    '엔트리ID', '원어 멤버ID', '표제어', '어깨번호', '표제어종류', '연관어', '연관단어 ID', '연관어 뜻 ID', '연관단어 ID.1',
    '연관어 표제어 ID'
]]
past_rel = past_rel[(past_rel != '').any(axis=1)]
past_rel.reset_index(inplace=True, drop=True)

In [85]:
past_rel.to_csv('../asset/ohw_id_rel.csv', index=False)

In [87]:
condition = present['표제어종류'] == '단어'
present = present[condition]
present.to_csv('../asset/id_present.csv', index=False)